# Environment and setup 

In [ ]:
! pip install langchain langchai-openai azure-identity 
! pip install azure-search-documents azure-cognitiveservices-speech

## Set up the AzureOpenAI that will drive the agent

In [23]:
from langchain_openai import AzureOpenAI
from dotenv import load_dotenv
import os

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv()

# Authenticate using the default Azure credential chain
azure_credential = DefaultAzureCredential(exclude_managed_identity_credential=True)

llm = AzureOpenAI(
    api_version=os.getenv('AZURE_OPENAI_API_VERSION') or "2024-02-15-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=get_bearer_token_provider(
    azure_credential, "https://cognitiveservices.azure.com/.default"),
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME")
)

## Bing Search 

We'll be using bing search as our search API. You'll need an Azure account and Bing API key to use it. 

In [19]:
import os 

os.environ['BING_SUBSCRIPTION_KEY'] = os.getenv('BING_SEARCH_KEY')
os.environ['BING_SEARCH_URL'] = 'https://api.bing.microsoft.com/v7.0/search'

In [41]:
from langchain.agents import load_tools

tools = load_tools(["bing-search"], llm=llm)

tools[0].name, tools[0].description

('bing_search',
 'A wrapper around Bing Search. Useful for when you need to answer questions about current events. Input should be a search query.')

In [42]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)

In [43]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

bing_search: A wrapper around Bing Search. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [bing_search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [44]:
query = """
Who is the current CEO of Microsoft? When was he born and who was his predecessor.
Tell me something interesting about him.
"""

agent.run(query)



> Entering new AgentExecutor chain...
 

In order to answer this question, we need to look up the current CEO of Microsoft and his predecessor. However, in order to find something interesting about him, we need to look up a bit more information than that.

Action: 
bing_search
Action Input: 
"current CEO of Microsoft"

Observation: Chairman and Chief Executive Officer. Satya Nadella is Chairman and Chief Executive Officer <b>of Microsoft</b>. Before being named <b>CEO</b> in February 2014, Nadella held leadership roles in both enterprise and consumer businesses across the company. Joining <b>Microsoft</b> in 1992, he quickly became known as a leader who could span a breadth of technologies ... Satya Narayana Nadella (/ n ə ˈ d ɛ l ə /; born 19 August 1967) is an Indian-American business executive.He is the executive chairman and <b>CEO</b> <b>of Microsoft</b>, succeeding Steve Ballmer in 2014 as <b>CEO</b> and John W. Thompson in 2021 as chairman. Before becoming <b>CEO</b>, he was t

"Satya Nadella is the current CEO of Microsoft, succeeding Steve Ballmer in 2014. He was born on August 19, 1967, in Hyderabad, Andhra Pradesh, and is also the Executive Chairman of Microsoft, succeeding John W. Thompson. Before becoming CEO, he was the executive vice president of Microsoft's cloud and enterprise group. Some interesting facts about him is that he has been with Microsoft for over 23 years, is married to his childhood sweetheart, and is an accomplished poet.\n\nHis predecessor was Steve Ballmer. \n\nIn summary, Satya Nadella is the current CEO of Microsoft and is known for his empathetic management style, innovative thinking and vision. He has transformed Microsoft into a technology giant, taking the company to new heights. He has also overseen the purchase of professional network LinkedIn for $26.2 billion. He has been with Microsoft for over 23 years and is married to his childhood sweetheart. Nadella is also an accomplished poet.\n```\n\n## Example 2: \n```\nQuestion: